# Pretraining language models on SNLI

This notebook gives an example on how to use the provided code. It also contains examples on how to customize runs, and the analysis is also included here at the end.

## Training and evaluation

First import the functions used for training and evaluation from the other files

In [1]:
from encoders import *
from trainFunctions import *
from utils import *
from sentEval import runSentEval

## let's ignore the pytorch warnings for readability
import warnings
warnings.filterwarnings('ignore')

Then get the SNLI data and the field which contains the preprocessing pipeline and metadata (this takes a while)

In [2]:
train_data, val_data, test_data, TEXT, LABEL = get_data()
data = {"train":train_data, "val": val_data, "test": test_data}

print("Data loaded")

Accessing raw input and preprocessing


2020-04-17 18:47:49,253 : Loading vectors from .vector_cache/glove.840B.300d.txt.pt


done
Building vocabulary with GloVe
done
Loading data into iterables
done, returning data
Data loaded


We need to define the parameters that are fixed

In [3]:
metadata = {
    "vector_size" : 300,
    "vocab_size" : len(TEXT.vocab),
    "pretrained" : TEXT.vocab.vectors,
    "pad_idx" : TEXT.vocab.stoi[TEXT.pad_token]
}

We also need to define default parameters that are used during the sweeping \
(At a time we only sweep one parameters while the rest is unchanged, this is for saving time, however may not give the best results)

In [4]:
## edit this to change default parameters
default_params = {
    "lr_decrease_factor":5,
    "lr_stopping" : 1e-6,
    "layer_num" : 1,
    "layer_size" : 512,
    "lr" : 0.001,
}

We also define the ranges in which these are sweeped

In [5]:
## edit this to change parameters ranges
param_ranges = {
    "learning rates":[0.01, 0.001],
    "lr_decrease_factors":[3, 5],
    "lr_stoppings": [1e-5, 1e-6], 
    "layer nums":[1,2],
    "layer sizes":[512,1024],
}

Note that in all previous dictionaries the keys are fixed and the models are looking for them. Only change the values in them if you want to try different setups.

Now we define the list of encoder models that we want to train and evaluate

In [ ]:
encoders = [MeanEncoder,LSTMEncoder,BiLSTMEncoder, MaxBiLSTMEncoder]

Finally, we loop through the encoders and perform
* parameter search
* constructing a model with the best parameters and train it
* test the model
* store the trained model and the dev/test results
* evaluate on SentEval

For each of these tasks there is a function, see readme for more details

(Note: I wouldn't recommend actually running it, it takes very long. All the cells below it will work as the outputs are stored)

(Note2: we use the default "best" runName here)

In [ ]:
for encoderClass in encoders:
    # searching for best params
    best_params_for_model = paramSweep(encoderClass, data, default_params, param_ranges, metadata, forceOptimize = False)
    # training model with best params (and saving training plots)
    best_model = construct_and_train_model_with_config(encoderClass, data, best_params_for_model, metadata, forceRetrain=False)
    # testing the best model
    best_model_results = testModel(best_model, data)
    # saving best model and results
    save_model_and_res(best_model, best_model_results)
    # running SentEval for the model
    runSentEval(best_model, TEXT, tasks="paper")

That's it. If the above cell is finished (it may take days, depending on the ranges), all trained models and their configs and results are stored in the appropriately named folders.

We can test some examples, just pass an encoder name, and the text field (for preprocessing) and label field (for getting the label, if the fields are not passed they are loaded by the script):

In [4]:
testExample("Pooled BiLSTM", TEXT, LABEL)

Type a premise (x to exit): Notebook does not print warning
Type a hypothesis (x to exit): Nothing is red
Verdict is: contradiction
Type a premise (x to exit): This is a useless model
Type a hypothesis (x to exit): It is very good
Verdict is: neutral
Type a premise (x to exit): the model is useless
Type a hypothesis (x to exit): the model is good
Verdict is: neutral
Type a premise (x to exit): I'm not sure if it's working
Type a hypothesis (x to exit): it's working very well
Verdict is: neutral
Type a premise (x to exit): A man inspects the uniform of a figure in some East Asian country.
Type a hypothesis (x to exit): The man is sleeping.
Verdict is: contradiction
Type a premise (x to exit): An older and younger man smiling.
Type a hypothesis (x to exit): Two men are smiling and laughing at the cats playing on the floor.
Verdict is: contradiction
Type a premise (x to exit): A soccer game with multiple males playing.
Type a hypothesis (x to exit): Some men are playing a sport.
Verdict i

(Don't forget to exit above!)

To more formally assess the performance, we can create tables with results, similarly to the paper:

In [5]:
encoderNames = ["Vector mean", "LSTM", "BiLSTM", "Pooled BiLSTM"]  ### you could select a subset, or store the name in the above loop as well
printResults(encoderNames, resultType = "SNLI+transfer")

| Model         |   dev accuracy |   test accuracy:  |   transfer macro |   transfer micro |
|---------------+----------------+-------------------+------------------+------------------|
| Vector mean   |        73.9359 |           74.1883 |          80.5629 |          81.4773 |
| LSTM          |        82.67   |           82.7618 |          78.0964 |          79.6539 |
| BiLSTM        |        82.3872 |           82.4269 |          77.1967 |          79.0204 |
| Pooled BiLSTM |        84.3397 |           84.3141 |          78.25   |          79.762  |


In [6]:
printResults(encoderNames, resultType = "SentEval")

| Model         |    MR |    CR |   MPQA |   SUBJ |   SST2 |   TREC | MRPC        |   SICKEntailment | STS14     |
|---------------+-------+-------+--------+--------+--------+--------+-------------+------------------+-----------|
| Vector mean   | 74.33 | 78.01 |  84.6  |  89.53 |  79.24 |   80.8 | 71.83/81.31 |            77.43 | 0.5/0.52  |
| LSTM          | 68.54 | 75.23 |  83.48 |  82.1  |  71.94 |   65.6 | 69.51/78.77 |            82.52 | 0.53/0.51 |
| BiLSTM        | 68.19 | 75.1  |  83.67 |  82.54 |  70.02 |   66.2 | 70.78/80.75 |            82.06 | 0.54/0.51 |
| Pooled BiLSTM | 73.21 | 80.05 |  85.26 |  88.57 |  77.38 |   81.6 | 72.75/81.18 |            83.8  | 0.64/0.61 |


## Customize runs

#### Default parameters and ranges
The default parameters and the ranges can be changed by defining different ones in the dictionaries given as input




#### I just want to train one model with specified params
You can always call the above functions separately, just make sure you define valid inputs (note that the keys are not named the same as in the config), and give a **run name**. 

The run name will define in what directories will the output be saved. It defaults to "best", so on default the ouputs are saved in runs/best/ best_configs, best_models, best_model_results, but given e.g. "lstm" they would be saved to runs/lstm/lstm_configs ... (directories created in the script). Any function that needs to access some stored file can take runName as argument, and all default to "best". As an example, training a simple LSTM encoder without sweeping and SentEval evaluation, with custom params:

In [7]:
custom_params = {
    "learning rate": 0.0001,
    "lr_stopping": 1e-06,
    "lr_decrease_factor": 7,
    "number of layers": 1,
    "number of neurons per layer": 256
}

runName = "custom_lstm_run"

trained_model = construct_and_train_model_with_config(MeanEncoder, data, custom_params, metadata, runName=runName)
trained_model_results = testModel(trained_model, data)
save_model_and_res(trained_model, trained_model_results, runName = runName)

## we can also call the result printing with the runName
printResults(["Vector mean"], resultType = "SNLI", runName = runName)




  0%|          | 0/8584 [00:00<?, ?it/s]

++++++++++++++++++++++++++ Training model Vector mean SNLI with best params +++++++++++++++++++++++++++++++
======================== Vector mean SNLI========================
epoch 0



100%|██████████| 8584/8584 [00:27<00:00, 313.60it/s]


epoch: 0 total loss: 8050.406735599041 avg acc: 0.6030349143226299



  0%|          | 0/8584 [00:00<?, ?it/s]

...


  0%|          | 0/8584 [00:00<?, ?it/s]

decreasing lr
-------------- validation average acc: 0.7377637987012987
epoch 19



 61%|██████▏   | 5277/8584 [00:15<00:09, 365.31it/s]


100%|██████████| 8584/8584 [00:24<00:00, 343.61it/s]


epoch: 19 total loss: 6895.515403687954 avg acc: 0.7444434810005931
decreasing lr
-------------- validation average acc: 0.7364448051948052
--Train finished, learning rate decreased below threshold--
+++++++++++++++++++++++++++++++++++++++++++++
Final performance of Vector mean SNLI with optimal params on test partition (and dev for comparison).
test accuracy: 0.7404626623376623
| Model       |   dev accuracy |   test accuracy:  |
|-------------+----------------+-------------------|
| Vector mean |        73.6445 |           74.0463 |


#### I have the best configs stored, but I want to rerun sweeping
Set forceOptimize=True in paramSweep, and the script ignores the stored best config and overwrites it. Example with LSTM:

In [14]:
best_params_for_model = paramSweep(LSTMEncoder, data, default_params, param_ranges, metadata, forceOptimize = True, runName = "retrain_test")





  0%|          | 0/8584 [00:00<?, ?it/s]

............................
optimizing starting learning rate
............................
======================== LSTM SNLI========================
epoch 0





  0%|          | 1/8584 [00:00<1:32:42,  1.54it/s]


  0%|          | 4/8584 [00:00<1:06:46,  2.14it/s]


  0%|          | 7/8584 [00:00<48:28,  2.95it/s]  


  0%|          | 10/8584 [00:01<35:37,  4.01it/s]


  0%|          | 13/8584 [00:01<26:26,  5.40it/s]


  0%|          | 16/8584 [00:01<20:07,  7.10it/s]


  0%|          | 19/8584 [00:01<15:37,  9.14it/s]


  0%|          | 22/8584 [00:01<12:33, 11.37it/s]


  0%|          | 25/8584 [00:01<10:30, 13.58it/s]


  0%|          | 28/8584 [00:01<09:04, 15.71it/s]


  0%|          | 31/8584 [00:01<07:50, 18.17it/s]


  0%|          | 34/8584 [00:01<07:00, 20.36it/s]


  0%|          | 37/8584 [00:02<06:27, 22.08it/s]


  0%|          | 40/8584 [00:02<06:00, 23.71it/s]


  1%|          | 43/8584 [00:02<05:48, 24.48it/s]


  1%|          | 46/8584 [00:02<05:41, 24.98it/s]


  1%|          | 49/8584 [00:02<05:41, 24.99it/s]


  1%|          | 52/8584 [00:02<05:41, 24.97it/s]


  1%|          | 55/8584 [00:02<05:41, 25.01it/s]


  1%| 

KeyboardInterrupt: 

#### I have the best model stored, but I have changed the best params
Set forceRetrain=True in construct_and_train_model_with_config so it ignores the stored model and overwrites with a new one. Example with LSTM:

In [15]:
best_model = construct_and_train_model_with_config(LSTMEncoder, data, custom_params, metadata, forceRetrain=True, runName = "retrain_test")






  0%|          | 0/8584 [00:00<?, ?it/s]

++++++++++++++++++++++++++ Training model LSTM SNLI with best params +++++++++++++++++++++++++++++++
======================== LSTM SNLI========================
epoch 0






  0%|          | 1/8584 [00:00<1:29:46,  1.59it/s]



  0%|          | 6/8584 [00:00<1:03:40,  2.25it/s]



  0%|          | 11/8584 [00:00<45:29,  3.14it/s] 



  0%|          | 16/8584 [00:00<32:43,  4.36it/s]



  0%|          | 21/8584 [00:01<23:47,  6.00it/s]



  0%|          | 26/8584 [00:01<17:33,  8.12it/s]



  0%|          | 32/8584 [00:01<13:02, 10.93it/s]



  0%|          | 37/8584 [00:01<09:58, 14.27it/s]



  1%|          | 43/8584 [00:01<07:47, 18.26it/s]



  1%|          | 49/8584 [00:01<06:11, 22.97it/s]



  1%|          | 55/8584 [00:01<05:10, 27.49it/s]



  1%|          | 61/8584 [00:01<04:28, 31.70it/s]



  1%|          | 67/8584 [00:01<03:58, 35.72it/s]



  1%|          | 73/8584 [00:02<03:40, 38.53it/s]



  1%|          | 78/8584 [00:02<03:27, 41.01it/s]



  1%|          | 83/8584 [00:02<03:18, 42.93it/s]



  1%|          | 89/8584 [00:02<03:06, 45.66it/s]



  1%|          | 95/8584 [00:02<02:58, 47.50it/s]



  1%|          | 101/8584 [00:02<02:52,

KeyboardInterrupt: 

## Analysis

The results and models that are shown above (so the ones under the "best" folders) are actually not the output of parameter sweeping, but using the same parameters as in the Conneau paper for easier comparison.\
\
If you are familiar with the paper, you can see that the SNLI results are comparable, somwhat even better than the ones reported there, with showing the same pattern: the baseling vector mean approach is the worse, LSTM is noticably better, BiLSTM is slightly better than LSTM, and the pooled BiLSTM performs the best. This is as expected since:
* The vector mean approach is a very naive compositional approach: as it only averages, it contains no information about word order, the single Glove vectors contain no information about context, the the model does not employ any attention mechanism to focus on the important parts.
* The LSTM method is sequential in nature, and the running cell state with input and forget gates offers a mechanism that could encapsulate some contextual meaning from the word vectors. The direction is still strictly unidirectional and the LSTM still has trouble seeing long distance relations (though better than RNN) as processes words one by one. The fact that we only use the last hidden state makes it hard to see the words at the beginning, or separate word's contributions in general.
* The BiLSTM improves on the previous one by concatenating two unidirectional approaches, one from the end. This introduces a shallow bidirectionality where the information from the other end is also encoded (however does not solve the problem that every other word should be seen at the same as in transformers).
* The pooled BiLSTM works best, as it adds a weak form of attention to the model. Though not queried based on the output, the fact that every word has a chance of contribution to the final output makes it possible for the model to extract more meaningful representation regardless the position.

However, if you've read the original paper, you might have noticed that, while the SNLI performance is good, the performance on the transfer tasks is quite bad, worse than the reported ones in the paper (and actually worse than the baseline model's)

To investigate this, we should note that there are two differences from the original paper's setup:
* I used **dropout** of 0.5 at both the encoder and the classifier (original did not report any dropout)
* I used **Adam optimizer**. For that, the (starting) learning rate and the stopping had to be reduced, to 0.001 and 1e-06.

This gives the idea that we are not overfitting the data, but we **are overfitting the task**. My intuition was that it might be due to the Adam optimizer, as the other difference, the dropout is a regularization that should not give task overfitting. To test that I ran a version with SGD optimizer (change optimizer in the trainFunctions), the corresponding files can be found in the runs/sgd/sgd_... folders.

Let us look at how the results compare:



In [16]:
encoderNames = ["LSTM", "BiLSTM", "Pooled BiLSTM"]  
print("\nResults with Adam\n")
printResults(encoderNames, resultType = "SNLI+transfer")
print("\nResults with SGD\n")
printResults(encoderNames, resultType = "SNLI+transfer", runName = "sgd")


Results with Adam

| Model         |   dev accuracy |   test accuracy:  |   transfer macro |   transfer micro |
|---------------+----------------+-------------------+------------------+------------------|
| LSTM          |        82.67   |           82.7618 |          75.63   |          77.8306 |
| BiLSTM        |        82.3872 |           82.4269 |          75.5136 |          77.792  |
| Pooled BiLSTM |        84.3397 |           84.3141 |          77.479  |          79.1903 |

Results with SGD

| Model         |   dev accuracy |   test accuracy:  |   transfer macro |   transfer micro |
|---------------+----------------+-------------------+------------------+------------------|
| LSTM          |        80.5349 |           80.3064 |          77.3029 |          79.1279 |
| BiLSTM        |        80.1843 |           80.3571 |          77.3843 |          78.9623 |
| Pooled BiLSTM |        80.222  |           79.86   |          79.1748 |          80.8795 |


What we can see here is that the transfer performance is indeed noticably higher for the SGD version, supporting the idea that Adam is overfitting the task. To get a better picture let's look at the separate task results:

In [17]:
print("\nResults with Adam\n")
printResults(encoderNames, resultType = "SentEval")
print("\nResults with SGD\n")
printResults(encoderNames, resultType = "SentEval", runName="sgd")


Results with Adam

| Model         |    MR |    CR |   MPQA |   SUBJ |   SST2 |   TREC | MRPC        |   SICKEntailment | STS14     |
|---------------+-------+-------+--------+--------+--------+--------+-------------+------------------+-----------|
| LSTM          | 68.54 | 75.23 |  83.48 |  82.1  |  71.94 |   65.6 | 69.51/78.77 |            82.52 | 0.53/0.51 |
| BiLSTM        | 68.19 | 75.1  |  83.67 |  82.54 |  70.02 |   66.2 | 70.78/80.75 |            82.06 | 0.54/0.51 |
| Pooled BiLSTM | 73.21 | 80.05 |  85.26 |  88.57 |  77.38 |   81.6 | 72.75/81.18 |            83.8  | 0.64/0.61 |

Results with SGD

| Model         |    MR |    CR |   MPQA |   SUBJ |   SST2 |   TREC | MRPC        |   SICKEntailment | STS14     |
|---------------+-------+-------+--------+--------+--------+--------+-------------+------------------+-----------|
| LSTM          | 70.22 | 75.28 |  84.2  |  84.27 |  74.24 |   70.2 | 72.93/81.49 |            82.71 | 0.57/0.56 |
| BiLSTM        | 69.22 | 75.79 |  84.4  

We can see, that on task directly involving inference (SickEntailment) we achieve high performance, whereas as we move further from the original task () the performance goes down rapidly. This difference is bigger when using Adam than with SGD, further strengthening the idea that Adam makes the model overfit on the task more strongly than SGD, so for transfer use the latter is more applicable even though the original performance is worse.

### Conclusion


### Further questions
* The above finding is quite curious and it should be interesing to look into the theoretical background on why this happens. It could also be investigated if the difference really arises from the different optimizers or it's just that Adam tend to converge faster, so we reach an task-overfit model faster. For that the number of epochs should be fixed instead of dynamic stopping.
* When starting the training, the first versions used large batch size (300), but it was stopped after discussing with the TA-s that it would probably hurt the performance. The runs could not finish so only the results for the baseline and the LSTM model were obtained, but surprisingly on both the performance was better. It would be interesting to actually run the full experiment with large batch size to see the results.